In [5]:
# pip install numpy==1.24

In [6]:
import numpy as np
import pandas as pd
import spacy
import nltk
from nltk.tokenize import sent_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from spacy import displacy
from spacy.matcher import PhraseMatcher
from nltk import pos_tag
from nltk.tokenize import word_tokenize
from spacy.lang.en import English

In [30]:
# Download necessary NLTK data
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kbeni\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\kbeni\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kbeni\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\kbeni\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\kbeni\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger_eng.zip.


True

In [8]:
data = pd.read_pickle('mergerdata.pkl')
# data = pd.read_json('merger_data.json')

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 825773 entries, 0 to 825772
Data columns (total 20 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   date_completion           506135 non-null  object
 1   bidders_name              825773 non-null  object
 2   date_completion_assumed   161144 non-null  object
 3   date_last_status_update   175350 non-null  object
 4   status                    825773 non-null  object
 5   bidders_industry          825773 non-null  object
 6   id                        825773 non-null  int64 
 7   comments                  825773 non-null  object
 8   targets_industry          825773 non-null  object
 9   targets_isin              825773 non-null  object
 10  date_rumor                825773 non-null  object
 11  date_postponed            214 non-null     object
 12  date_announcement         696242 non-null  object
 13  bidders_isin              825773 non-null  object
 14  date

In [10]:
# filter data from before 2010-01-01
data = data[data['date_rumor'] >= '2010-01-01']

In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 519721 entries, 1 to 825772
Data columns (total 20 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   date_completion           302059 non-null  object
 1   bidders_name              519721 non-null  object
 2   date_completion_assumed   108961 non-null  object
 3   date_last_status_update   103980 non-null  object
 4   status                    519721 non-null  object
 5   bidders_industry          519721 non-null  object
 6   id                        519721 non-null  int64 
 7   comments                  519721 non-null  object
 8   targets_industry          519721 non-null  object
 9   targets_isin              519721 non-null  object
 10  date_rumor                519721 non-null  object
 11  date_postponed            104 non-null     object
 12  date_announcement         434568 non-null  object
 13  bidders_isin              519721 non-null  object
 14  date_comp

In [12]:
# filter industries by keywords
industries_data = pd.read_csv('industries.csv')

keywords_auto_oil = ["automobile", "vehicle", "car", "motor", "engine", "oil", "gas", "fuel", "petroleum"]
relevant_rows = industries_data[
    industries_data['description'].str.contains('|'.join(keywords_auto_oil), case=False, na=False)
]

# Extract the unique codes for these descriptions
relevant_codes = relevant_rows['code'].unique()

# change relevant_codes to strings
relevant_codes = [str(code) for code in relevant_codes]

relevant_codes = set(relevant_codes)

In [13]:
import ast


# Function to check if any relevant code is included
def compare_if_included(industries):
    if not industries:  # Handle empty or None values gracefully
        return False
    try:
        # Parse the entry if it's a string representation of a list
        if isinstance(industries, str):
            industries = ast.literal_eval(industries)
        # Check if any industry code matches the relevant codes
        return any(str(industry) in relevant_codes for industry in industries)
    except (ValueError, SyntaxError, TypeError):
        # Return False if parsing fails or industries is malformed
        return False

# Apply the function to the 'targets_industry' column
data['has_relevant_code'] = data['targets_industry'].apply(compare_if_included)

In [14]:
# Filter data to only include rows with relevant codes
data = data[data['has_relevant_code']]

In [15]:
data.drop(columns=['has_relevant_code'], inplace=True)

In [39]:
data

,date_completion,bidders_name,date_completion_assumed,date_last_status_update,status,bidders_industry,id,comments,targets_industry,targets_isin,...,targets_country,type,targets_name,bidders_country,sentences,cleaned_sentences,stemmed_sentences,lemmatized_sentences,pos_tagged_sentences,dependency_parsed_sentences
17,2014-07-31,[jaya hind investments pvt ltd],None,None,Completed,[6499],1907056319,2014-08-01: it was announced that on 31/07/14 ...,[2910],[INE451A01017],...,[IN],Acquisition 3.188%,[force motors ltd],[IN],[2014-08-01: it was announced that on 31/07/14...,[20140801 it was announced that on 310714 jaya...,[20140801 it wa announc that on 310714 jaya hi...,[20140801 it be announce that on 310714 jaya h...,"[[(20140801, CD), (it, PRP), (was, VBD), (anno...","[[(20140801, npadvmod, announced), (it, nsubjp..."
119,2017-07-21,"[guangzhou trauer biotechnology co., ltd]",None,2017-07-25,Completed,[2120],1907184442,2016-11-24: it was announced that Guangzhou Tr...,[2042],[None],...,[CN],Acquisition increased from 100% to 100%,"[guangzhou chuangermei biotechnology co., ltd]",[CN],[2016-11-24: it was announced that Guangzhou T...,[20161124 it was announced that guangzhou trau...,[20161124 it wa announc that guangzhou trauer ...,[20161124 it be announce that guangzhou trauer...,"[[(20161124, CD), (it, PRP), (was, VBD), (anno...","[[(20161124, npadvmod, announced), (it, nsubjp..."
143,None,[alconix corporation],2020-12-03,None,Completed Assumed,[4672],1941467148,2020-11-25: it was announced that Alconix Corp...,[2521],[None],...,[JP],Acquisition increased from 3% to 95%,"[fujine sangyo co., ltd]",[JP],[2020-11-25: it was announced that Alconix Cor...,[20201125 it was announced that alconix corpor...,[20201125 it wa announc that alconix corpor ha...,[20201125 it be announce that alconix corporat...,"[[(20201125, CD), (it, PRP), (was, VBD), (anno...","[[(20201125, npadvmod, announced), (it, nsubjp..."
161,None,"[tianjin winspirit energy technology co., ltd]",2019-06-01,None,Completed Assumed,[7219],1907220901,2017-06-01: it was announced that Tianjin Wins...,[7219],[None],...,[CN],Acquisition increased from 100% to 100%,"[guizhou weiyuan technology co., ltd]",[CN],[2017-06-01: it was announced that Tianjin Win...,[20170601 it was announced that tianjin winspi...,[20170601 it wa announc that tianjin winspirit...,[20170601 it be announce that tianjin winspiri...,"[[(20170601, CD), (it, PRP), (was, VBD), (anno...","[[(20170601, npadvmod, announced), (it, nsubjp..."
163,None,"[tenneco automotive (thailand) co., ltd]",2013-06-30,None,Completed Assumed,[2932],1633043238,2011-06-30: it was announced that Yarnapund PC...,[2932],[None],...,[TH],Acquisition increased from 75% to 100%,"[walker exhaust (thailand) co., ltd]",[TH],[2011-06-30: it was announced that Yarnapund P...,[20110630 it was announced that yarnapund pcl ...,[20110630 it wa announc that yarnapund pcl ha ...,[20110630 it be announce that yarnapund pcl ha...,"[[(20110630, CD), (it, PRP), (was, VBD), (anno...","[[(20110630, npadvmod, announced), (it, nsubjp..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
825710,None,[vestum ab],2021-10-05,None,Completed Assumed,[4399],1941588290,2021-09-15: it was announced that Vestum AB wi...,"[4299, 4299]","[None, None]",...,"[SE, SE]",Acquisition 100%,"[hanell entreprenad i gavle ab, per lennartsso...",[SE],[2021-09-15: it was announced that Vestum AB w...,[20210915 it was announced that vestum ab will...,[20210915 it wa announc that vestum ab will ac...,[20210915 it be announce that vestum ab will a...,"[[(20210915, CD), (it, PRP), (was, VBD), (anno...","[[(20210915, npadvmod, announced), (it, nsubjp..."
825721,None,"[shanghai zj bio-tech co., ltd]",2019-02-08,None,Completed Assumed,[2120],1907197166,2017-02-08: it was announced that Shanghai ZJ ...,[7219],[None],...,[CN],Acquisition increased from 100% to 100%,"[shanghai zj biomedical technology co., ltd]",[CN],[2017-02-08: it was announced that 

In [17]:
# Step 3: Sentence Tokenization (using NLTK)
data['sentences'] = data['comments'].apply(lambda x: sent_tokenize(x))

In [18]:
# Step 4: Lowercasing and Removing Punctuation
def clean_text(text):
    # Convert to lowercase
    text = text.lower()
    
    # Remove punctuation (SpaCy already handles punctuation during processing)
    text = ''.join([char for char in text if char.isalnum() or char.isspace()])
    
    return text

In [19]:
# Apply cleaning function to each sentence
data['cleaned_sentences'] = data['sentences'].apply(lambda sentences: [clean_text(sentence) for sentence in sentences])

In [20]:
# Step 5: Stemming (optional, more aggressive than Lemmatization)
def apply_stemming(sentence):
    ps = PorterStemmer()
    words = word_tokenize(sentence)
    return ' '.join([ps.stem(word) for word in words])

In [21]:
# Apply stemming to each sentence
data['stemmed_sentences'] = data['cleaned_sentences'].apply(lambda sentences: [apply_stemming(sentence) for sentence in sentences])

In [22]:
spacy.cli.download("en_core_web_sm")

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [23]:
nlp = spacy.load('en_core_web_sm')

In [24]:
# Step 6: Lemmatization using SpaCy
def lemmatize_sentence(sentence):
    doc = nlp(sentence)
    return ' '.join([token.lemma_ for token in doc])

In [25]:
# Apply Lemmatization
data['lemmatized_sentences'] = data['cleaned_sentences'].apply(lambda sentences: [lemmatize_sentence(sentence) for sentence in sentences])

In [26]:
data.head()

,date_completion,bidders_name,date_completion_assumed,date_last_status_update,status,bidders_industry,id,comments,targets_industry,targets_isin,...,date_completion_expected,date_withdrawn,targets_country,type,targets_name,bidders_country,sentences,cleaned_sentences,stemmed_sentences,lemmatized_sentences
17,2014-07-31,[jaya hind investments pvt ltd],None,None,Completed,[6499],1907056319,2014-08-01: it was announced that on 31/07/14 ...,[2910],[INE451A01017],...,None,None,[IN],Acquisition 3.188%,[force motors ltd],[IN],[2014-08-01: it was announced that on 31/07/14...,[20140801 it was announced that on 310714 jaya...,[20140801 it wa announc that on 310714 jaya hi...,[20140801 it be announce that on 310714 jaya h...
119,2017-07-21,"[guangzhou trauer biotechnology co., ltd]",None,2017-07-25,Completed,[2120],1907184442,2016-11-24: it was announced that Guangzhou Tr...,[2042],[None],...,None,None,[CN],Acquisition increased from 100% to 100%,"[guangzhou chuangermei biotechnology co., ltd]",[CN],[2016-11-24: it was announced that Guangzhou T...,[20161124 it was announced that guangzhou trau...,[20161124 it wa announc that guangzhou trauer ...,[20161124 it be announce that guangzhou trauer...
143,None,[alconix corporation],2020-12-03,None,Completed Assumed,[4672],1941467148,2020-11-25: it was announced that Alconix Corp...,[2521],[None],...,2020-12-03,None,[JP],Acquisition increased from 3% to 95%,"[fujine sangyo co., ltd]",[JP],[2020-11-25: it was announced that Alconix Cor...,[20201125 it was announced that alconix corpor...,[20201125 it wa announc that alconix corpor ha...,[20201125 it be announce that alconix corporat...
161,None,"[tianjin winspirit energy technology co., ltd]",2019-06-01,None,Completed Assumed,[7219],1907220901,2017-06-01: it was announced that Tianjin Wins...,[7219],[None],...,None,None,[CN],Acquisition increased from 100% to 100%,"[guizhou weiyuan technology co., ltd]",[CN],[2017-06-01: it was announced that Tianjin Win...,[20170601 it was announced that tianjin winspi...,[20170601 it wa announc that tianjin winspirit...,[20170601 it be announce that tianjin winspiri...
163,None,"[tenneco automotive (thailand) co., ltd]",2013-06-30,None,Completed Assumed,[2932],1633043238,2011-06-30: it was announced that Yarnapund PC...,[2932],[None],...,None,None,[TH],Acquisition increased from 75% to 100%,"[walker exhaust (thailand) co., ltd]",[TH],[2011-06-30: it was announced that Yarnapund P...,[20110630 it was announced that yarnapund pcl ...,[20110630 it wa announc that yarnapund pcl ha ...,[20110630 it be announce that yarnapund pcl ha...


In [27]:
for text in data['lemmatized_sentences']:
    print(text)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



['20221005 it be state that n trade doo have make an informal offer to acquire the remain 63609 per cent stake represent by 1193943 share in energoinv automatika ad a lukavica bosnia and herzegovinabase machinery and equipment renting service provider at bam 0255 per share', 'n trade currently hold a 36391 per cent stake represent by 683050 share in energoinv automatika', 'the offer price represent a rumour bid premium of 2 per cent over target closing share price of bam 025 per share on 041022 the last trading day prior the statement', 'the offer price value the transaction at bam 304455', '20221007 it be announce that komisija za hartije od vrjednosti republike srpske have approve the transaction', 'the offer price represent an announce bid premium of 2 per cent over target closing share price of bam 025 per share on 061022 the last trading day prior the announcement', '20221208 it be announce that the bid be publish and be valid for the next 30 day', '20221220 it be announce that ta

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [28]:
# Step 7: Part-of-Speech (POS) Tagging using NLTK (optional, more for understanding structure)
def pos_tagging(sentence):
    return pos_tag(word_tokenize(sentence))

In [31]:
# Apply POS tagging
data['pos_tagged_sentences'] = data['cleaned_sentences'].apply(lambda sentences: [pos_tagging(sentence) for sentence in sentences])

In [32]:
# Step 8: Dependency Parsing using SpaCy
def dependency_parsing(sentence):
    doc = nlp(sentence)
    return [(token.text, token.dep_, token.head.text) for token in doc]

In [33]:
# Apply Dependency Parsing
data['dependency_parsed_sentences'] = data['cleaned_sentences'].apply(lambda sentences: [dependency_parsing(sentence) for sentence in sentences])

In [34]:
data.head()

,date_completion,bidders_name,date_completion_assumed,date_last_status_update,status,bidders_industry,id,comments,targets_industry,targets_isin,...,targets_country,type,targets_name,bidders_country,sentences,cleaned_sentences,stemmed_sentences,lemmatized_sentences,pos_tagged_sentences,dependency_parsed_sentences
17,2014-07-31,[jaya hind investments pvt ltd],None,None,Completed,[6499],1907056319,2014-08-01: it was announced that on 31/07/14 ...,[2910],[INE451A01017],...,[IN],Acquisition 3.188%,[force motors ltd],[IN],[2014-08-01: it was announced that on 31/07/14...,[20140801 it was announced that on 310714 jaya...,[20140801 it wa announc that on 310714 jaya hi...,[20140801 it be announce that on 310714 jaya h...,"[[(20140801, CD), (it, PRP), (was, VBD), (anno...","[[(20140801, npadvmod, announced), (it, nsubjp..."
119,2017-07-21,"[guangzhou trauer biotechnology co., ltd]",None,2017-07-25,Completed,[2120],1907184442,2016-11-24: it was announced that Guangzhou Tr...,[2042],[None],...,[CN],Acquisition increased from 100% to 100%,"[guangzhou chuangermei biotechnology co., ltd]",[CN],[2016-11-24: it was announced that Guangzhou T...,[20161124 it was announced that guangzhou trau...,[20161124 it wa announc that guangzhou trauer ...,[20161124 it be announce that guangzhou trauer...,"[[(20161124, CD), (it, PRP), (was, VBD), (anno...","[[(20161124, npadvmod, announced), (it, nsubjp..."
143,None,[alconix corporation],2020-12-03,None,Completed Assumed,[4672],1941467148,2020-11-25: it was announced that Alconix Corp...,[2521],[None],...,[JP],Acquisition increased from 3% to 95%,"[fujine sangyo co., ltd]",[JP],[2020-11-25: it was announced that Alconix Cor...,[20201125 it was announced that alconix corpor...,[20201125 it wa announc that alconix corpor ha...,[20201125 it be announce that alconix corporat...,"[[(20201125, CD), (it, PRP), (was, VBD), (anno...","[[(20201125, npadvmod, announced), (it, nsubjp..."
161,None,"[tianjin winspirit energy technology co., ltd]",2019-06-01,None,Completed Assumed,[7219],1907220901,2017-06-01: it was announced that Tianjin Wins...,[7219],[None],...,[CN],Acquisition increased from 100% to 100%,"[guizhou weiyuan technology co., ltd]",[CN],[2017-06-01: it was announced that Tianjin Win...,[20170601 it was announced that tianjin winspi...,[20170601 it wa announc that tianjin winspirit...,[20170601 it be announce that tianjin winspiri...,"[[(20170601, CD), (it, PRP), (was, VBD), (anno...","[[(20170601, npadvmod, announced), (it, nsubjp..."
163,None,"[tenneco automotive (thailand) co., ltd]",2013-06-30,None,Completed Assumed,[2932],1633043238,2011-06-30: it was announced that Yarnapund PC...,[2932],[None],...,[TH],Acquisition increased from 75% to 100%,"[walker exhaust (thailand) co., ltd]",[TH],[2011-06-30: it was announced that Yarnapund P...,[20110630 it was announced that yarnapund pcl ...,[20110630 it wa announc that yarnapund pcl ha ...,[20110630 it be announce that yarnapund pcl ha...,"[[(20110630, CD), (it, PRP), (was, VBD), (anno...","[[(20110630, npadvmod, announced), (it, nsubjp..."


In [35]:
data.to_json('prepro__mergerdata.json')

In [36]:
data.to_csv('prepro_mergerdata.csv')

In [37]:
for bidder in data['bidders_name']:
    print(bidder)

['jaya hind investments pvt ltd']
['guangzhou trauer biotechnology co., ltd']
['alconix corporation']
['tianjin winspirit energy technology co., ltd']
['tenneco automotive (thailand) co., ltd']
['miro radici family of companies spa']
['daechang forging co., ltd']
['patec precision industry co., ltd']
['grupa kapitalowa immobile sa']
['guangdong yue hai water investment co., ltd']
['china automotive holdings ltd']
['billion legend co., ltd']
['beijing starneto technology co., ltd']
['an trung industries co., ltd']
['burkhalter holding ag']
['senci electric machinery co., ltd']
['yokohama rubber co., ltd, the']
['robostar co., ltd']
['bbr holdings (s) ltd']
['chongqing landai powertrain corporation ltd']
['jetson development (asia) sdn bhd']
['dongguan meixin business consulting co., ltd']
['shaanxi sunfonda automobile technology development co., ltd']
['beijing new building materials public co., ltd']
[]
['medical data vision co., ltd']
['chongqing machinery & electric co., ltd']
[]
['u

In [38]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 36135 entries, 17 to 825761
Data columns (total 26 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   date_completion              22097 non-null  object
 1   bidders_name                 36135 non-null  object
 2   date_completion_assumed      7857 non-null   object
 3   date_last_status_update      7328 non-null   object
 4   status                       36135 non-null  object
 5   bidders_industry             36135 non-null  object
 6   id                           36135 non-null  int64 
 7   comments                     36135 non-null  object
 8   targets_industry             36135 non-null  object
 9   targets_isin                 36135 non-null  object
 10  date_rumor                   36135 non-null  object
 11  date_postponed               3 non-null      object
 12  date_announcement            31677 non-null  object
 13  bidders_isin                 36135